In [ ]:
import os, getpass
from dotenv import load_dotenv 
load_dotenv(override=True)

In [ ]:
from typing import TypedDict, Annotated, Optional
from langgraph.graph import StateGraph, END
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate
import gradio as gr

from langchain_community.utilities import OpenWeatherMapAPIWrapper
weather = OpenWeatherMapAPIWrapper()

In [ ]:
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("GROQ API Key: ")

   #Check for Groq API Key
if "OPENWEATHERMAP_API_KEY" not in os.environ:
    os.environ["OPENWEATHERMAP_API_KEY"] = getpass.getpass("WEATHER API Key: ")


if "AMADEUS_CLIENT_ID" not in os.environ:
    os.environ["AMADEUS_CLIENT_ID"] = getpass.getpass("WEATHER API Key: ")
    
if "AMADEUS_CLIENT_SECRET" not in os.environ:
    os.environ["AMADEUS_CLIENT_SECRET"] = getpass.getpass("WEATHER API Key: ")    

In [ ]:
from langchain.chat_models import init_chat_model

model_name = "llama-3.1-8b-instant"
llm = init_chat_model(model_name, model_provider="groq") #Other Llama alternatives available are llama3-8b-8192, llama-3.3-70b-versatile

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage

llm = ChatOllama(
    model="llama3.1:8b",
    base_url="http://localhost:11434"  # 👈 this sets your Ollama server URL
)

In [ ]:
from pydantic import BaseModel
from typing import Optional, Annotated
from langchain_core.messages import BaseMessage
from langgraph.graph import MessagesState

class ReviewState(TypedDict, total=False):
    user_message: Annotated[str, "The original travel location input from the user"]
    planner_agent_feedback: Annotated[list[BaseMessage], "Feedback from the itinerary planner agent"]
    budget_agent_feedback: Annotated[list[BaseMessage], "Feedback from the budget agent"] 
   


In [ ]:
weather.run("Bali, Indonesia")

In [ ]:
from langchain.tools import Tool


def return_weather_data(user_inp: str) -> str:
    """Find weather of the Destination provided

    Args:
        user_inp: Destination user wants to visit

    """
    data = weather.run(user_inp)
    #print(data)
 
    return data # just return raw weather string


tools_P =  [Tool.from_function(
    name="return_weather_data",  # Name that LLM returns
    func=return_weather_data,
    description="Use this tool to get the current weather for a specific city. The input must be a single city name."
)]

# tools_P = [return_weather_data]

# llm_with_weather = llm.bind_tools(tools_P)

# output = llm_with_weather.invoke([HumanMessage(content="Trip to Bali")])
# output


In [ ]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain.prompts import PromptTemplate

prompt_p = PromptTemplate.from_template(
"""You are an Expert Itinerary Planning Agent. Your goal is to create a detailed travel itinerary for 1 week Max Dont give beyond that and should be less than 1800 characters.

You have access to the following tools:
{tools}

To use a tool, follow this strict format:

Thought: Your reasoning process, leading to a tool call or final answer.
Action: the name of the tool to call, ONLY one of [{tool_names}]
Action Input: the input parameters for the tool (e.g., "Mumbai")
Observation: the result received from the tool


Once you have gathered all necessary information (like weather for all cities) and are ready to present the itinerary, you MUST use the following format for your final response:

Thought: I have gathered all necessary information and I am now ready to present the detailed itinerary.
Final Answer: [Your complete and detailed travel itinerary, incorporating all weather data and logical planning.]


---
**Your Planning Steps:**

1.  **Understand the Request & Identify Cities (Mental Step):** Read the user's question and determine the primary cities for the itinerary (e.g., "Mumbai" and "London" from "Mumbai to London"). This is a thinking step; do NOT use an `Action:` for this.
2.  **Get Weather Data for Each City (Tool Use):** For EACH identified city, use the `return_weather_data` tool. Call it one city at a time.
    * **Example Action for weather:** `Action: return_weather_data\nAction Input: Mumbai`
3.  **Synthesize Information & Plan Itinerary (Mental Step):** Once you have the weather data for all cities, use your expertise to synthesize this information and logically plan the itinerary. This is an internal reasoning step; do NOT use an `Action:` for this.
4.  **Provide Final Answer (Output Format):** Present the complete itinerary using the `Final Answer` format described above.

---
**Important Constraint:**
- The user mentioned a budget, but you should IGNORE it as per your instructions.

Begin!

Question: {user_message}

{agent_scratchpad}
"""

)

# Step 5: Create agent and executor
agent = create_react_agent(llm=llm, tools=tools_P, prompt=prompt_p)
agent_exec = AgentExecutor(agent=agent, tools=tools_P, verbose=True,handle_parsing_errors=True,max_iterations=4)


In [ ]:
# LangChain's create_react_agent() requires these 3 input variables to be present in your PromptTemplate:

# {tools} – A description of each tool

# {tool_names} – A comma-separated list of tool names

# {agent_scratchpad} – Internal scratchpad for ReAct steps


#output = agent_exec.invoke({"user_message":"I want a vacation under $10000 for Mumbai to London?"})

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.messages import HumanMessage,SystemMessage

def planner_agent(state: ReviewState) -> ReviewState:

    print("STATE RECEIVED:", state)

    user_message = state.get("user_message", "testingggg")
    if not user_message:
        raise ValueError("Missing 'user_message' in state.")

    # The prompt expects a dict with only 'user_message'
    
    agent_resp = agent_exec.invoke({"user_message":user_message})
    
    print(agent_resp)
    # Return updated state
    return {
        "planner_agent_feedback": agent_resp["output"]
    }

In [ ]:

from amadeus import Client, ResponseError

amadeus_client = Client(
    client_id=os.environ["AMADEUS_CLIENT_ID"],
    client_secret=os.environ["AMADEUS_CLIENT_SECRET"]
)


from langchain.tools import Tool

def return_search_flights_tool(flight_information: str) -> str:
    """Search flights using input

    Always use the format for (BOM,DPS,2025-08-01).
    No Other Input should be added besides the three input with specific format
    """
    print("flight_information raw:", repr(flight_information))
    try:
        # Strip and split
        query_with_IATA_airport_codes_src, query_with_IATA_airport_codes_des, departure_date = map(str.strip, flight_information.strip().split(","))

        print(query_with_IATA_airport_codes_src)
        print(query_with_IATA_airport_codes_des)
        print(departure_date)

        response = amadeus_client.shopping.flight_offers_search.get(
            originLocationCode=query_with_IATA_airport_codes_src.upper(),
            destinationLocationCode=query_with_IATA_airport_codes_des.upper(),
            departureDate=departure_date,
            adults=1
        )

        return str(response.data[:3])  # show top 3 offers
    except ResponseError as e:
        return f"Amadeus API Error: {e} - {e.response.body}"
    except Exception as e:
        return f"Invalid input. Use format: 'BOM,DPS,2025-08-01'. Error: {str(e)}"


tools_B =  [Tool.from_function(
    name="return_search_flights_tool",  # Name that LLM returns
    func=return_search_flights_tool,
    description=  """  
    Use this tool to search for flight offers.
    The input to this tool must be a single string, containing three comma-separated values:
    Example Input: BOM,DPS,2025-08-01
    No Other Input should be added besides the three input with specific format
    """
)]



#

# llm_with_flight = llm.bind_tools(tools_B)

# output = llm_with_flight.invoke([HumanMessage(content="Trip form Mumbai to Bali under 10000 budget")])
# output

In [ ]:
return_search_flights_tool("BOM,DPS,2025-08-01")

In [ ]:


prompt_b = PromptTemplate.from_template(
 """
You are an expert **Budget Agent**. Your task is to select cost-effective **one-way** flights for a round trip using the destination and travel duration provided by the Planner Agent.

You have access to the following tool:
{tools}

You MUST follow this strict format:

Thought: Your reasoning process.
Action: The name of the tool to call, ONLY one of [{tool_names}]
Action Input: "source,destination,departure_date" (e.g., "BOM,LHR,2025-08-01")

Observation: The result received from the tool.

---

**Instructions:**

1. Perform **two one-way flight searches**:
    - First: from the source to the destination (e.g., BOM → LHR).
    - Second: from the destination back to the source (e.g., LHR → BOM), based on the return duration (e.g., 7 days after departure).
2. Use the **Planner Message** to extract:
    - The departure date.
    - The source and destination.
    - The number of days for the round trip.
3. Use the tool `return_search_flights_tool` only once per leg of the journey.
4. After the **first** search, summarize the flight options briefly.
5. Then, search for the **return** leg using the calculated return date.
6. If the tool returns an error, retry **only if** it makes sense. Do **NOT** use the same input again if it previously failed.
7. **Never call undefined tools or create your own tools.** Do **not** write code.
8. **Do not hallucinate results. Only respond based on actual tool output.**

---

**Final Answer Format:**

After evaluating both flights:

Thought: I have searched both flights and evaluated their prices.
Final Answer: in_budget or out_of_budget  
[Short summary including price, routes, dates, and Planner Message info]

---

Planner Message: 
{planner_agent_feedback}

Begin!




{agent_scratchpad}
"""

)

agent_B = create_react_agent(llm=llm, tools=tools_B, prompt=prompt_b)
agent_exec_b = AgentExecutor(agent=agent_B, tools=tools_B, verbose=True,handle_parsing_errors=True)


In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.messages import HumanMessage,AIMessage

def budget_agent(state: ReviewState) -> ReviewState:
    

    # Define the budget planner prompt
   

    msg_planner_agent_feedback = state.get("planner_agent_feedback","")

    print(msg_planner_agent_feedback)
    # Call the LLM with flight tool
    agent_resp = agent_exec_b.invoke({"planner_agent_feedback": str(msg_planner_agent_feedback)})
    print(agent_resp)

    # Update state with feedback
    #state["budget_agent_feedback"] = output

    return {"budget_agent_feedback":agent_resp["output"]}


In [ ]:
#agent_exec_b.invoke({"planner_agent_feedback":"I want a vacation under $10000 for Mumbai to Bali for date 1st august 2025?"})

In [ ]:
from typing import Literal

def budget_router(state: ReviewState) -> Literal["Planner_Agent", "Default"]:
    # Inspect state to decide

    print(state["user_message"])
    print(state["planner_agent_feedback"])
    print(state["budget_agent_feedback"])

    latest_message = state["budget_agent_feedback"]

    if "in_budget" in latest_message.lower():
        print("Routed to Default - END via message content")
        return "Default"

   
    print("Routed to Planner_Agent")
        
    return "Planner_Agent"

In [ ]:
from langgraph.graph import START,END, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode
from IPython.display import Image, display

graph_builder = StateGraph(ReviewState)

# Add only actual state-changing nodes
graph_builder.add_node("Planner_Agent", planner_agent)
graph_builder.add_node("Budget_Agent", budget_agent)


# Start at Planner_Agent
graph_builder.add_edge(START, "Planner_Agent")

graph_builder.add_edge("Planner_Agent", "Budget_Agent")


# After Budget_Agent, route using budget_router
graph_builder.add_conditional_edges(
    "Budget_Agent",
    budget_router,
    {
        "Planner_Agent": "Planner_Agent",
        "Default": END
    }
)


# Compile and visualize
graph = graph_builder.compile()
display(Image(graph.get_graph(xray=True).draw_mermaid_png()))


In [ ]:
result = graph.invoke({"user_message": "I want a vacation under budget of $10000 for Mumbai to London"})

for key, value in result.items():
    print(f"\n🔹 {key.upper()}:\n{value}")

In [ ]:
# --- Gradio Interface ---
def review(input_str: str) -> str:
    initial_state: ReviewState = {"user_message": input_str}
    final_state = graph.invoke(initial_state)
    return final_state['budget_agent_feedback']

iface = gr.Interface(
    fn=review,
    inputs=gr.Textbox(lines=10, label="Enter Trip Details"),
    outputs=gr.Textbox(lines=10, label="Planned Output"),
    title="Multi-Agent Trip Assistant",
    description="Submit Your trip request and get details  and plan from an LLM-powered agent pipeline."
)

In [ ]:
if __name__ == "__main__":
    iface.launch()